In [1]:
!cp -r ../input/recsys-repo/RecSys_Course_AT_PoliMi-master/* ./

# Loading Data

In [2]:
%config Completer.use_jedi = False
import pandas as pd
import numpy as np
import scipy.sparse as sps
import matplotlib.pyplot as pyplot
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample

In [3]:
#load data_train, matrix of users interaction
URM_path = "../input/recommender-system-2021-challenge-polimi/data_train.csv"
URM_all_dataframe = pd.read_csv(filepath_or_buffer=URM_path, 
                                sep=",",
                                dtype={0:int, 1:int, 2:float},
                                header=0)
URM_all_dataframe.columns = ["UserID", "ItemID", "Interaction"]

In [4]:
URM_all_dataframe.head()

,UserID,ItemID,Interaction
0,0,53,1.0
1,0,209,1.0
2,0,223,1.0
3,0,249,1.0
4,0,435,1.0


In [5]:
userID_unique = URM_all_dataframe["UserID"].unique()
itemID_unique = URM_all_dataframe["ItemID"].unique()

n_users = len(userID_unique)
n_items = len(itemID_unique)
n_interactions = len(URM_all_dataframe)

print ("Number of items\t {}, Number of users\t {}".format(n_items, n_users))
print ("Max ID items\t {}, Max Id users\t {}\n".format(max(itemID_unique), max(userID_unique)))
print ("Average interactions per user {:.2f}".format(n_interactions/n_users))
print ("Average interactions per item {:.2f}\n".format(n_interactions/n_items))

print ("Sparsity {:.2f} %".format((1-float(n_interactions)/(n_items*n_users))*100))

Number of items	 18059, Number of users	 13650
Max ID items	 18058, Max Id users	 13649

Average interactions per user 387.23
Average interactions per item 292.69

Sparsity 97.86 %


In [6]:
URM_all = sps.coo_matrix((URM_all_dataframe["Interaction"].values, 
                          (URM_all_dataframe["UserID"].values, URM_all_dataframe["ItemID"].values)))
URM_all = URM_all.tocsr() # to obtain fast access to rows (users)
URM_all

<13650x18059 sparse matrix of type '<class 'numpy.float64'>'
	with 5285664 stored elements in Compressed Sparse Row format>

# Data processing and basic tuning setup

In [7]:
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample

# split data into train and validation data 80/20
URM_train, URM_valid = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.80)

In [8]:
from Evaluation.Evaluator import EvaluatorHoldout

#create an evaluator object to evaluate validation set
#we will use it for hyperparameter tuning
evaluator_valid = EvaluatorHoldout(URM_valid, cutoff_list=[10])

EvaluatorHoldout: Ignoring 13640 ( 0.1%) Users that have less than 1 test interactions


In [9]:
from Recommenders.MatrixFactorization.IALSRecommender import IALSRecommender

baseline = IALSRecommender(URM_train)
baseline.fit(num_factors = 48, confidence_scaling= 'linear', alpha = 1.0, epsilon = 0.10715471885641545, reg = 0.0005294631576919714, epochs = 30)

IALSRecommender: Epoch 1 of 30. Elapsed time 10.88 sec
IALSRecommender: Epoch 2 of 30. Elapsed time 22.88 sec
IALSRecommender: Epoch 3 of 30. Elapsed time 33.63 sec
IALSRecommender: Epoch 4 of 30. Elapsed time 44.47 sec
IALSRecommender: Epoch 5 of 30. Elapsed time 56.43 sec
IALSRecommender: Epoch 6 of 30. Elapsed time 1.12 min
IALSRecommender: Epoch 7 of 30. Elapsed time 1.31 min
IALSRecommender: Epoch 8 of 30. Elapsed time 1.51 min
IALSRecommender: Epoch 9 of 30. Elapsed time 1.69 min
IALSRecommender: Epoch 10 of 30. Elapsed time 1.87 min
IALSRecommender: Epoch 11 of 30. Elapsed time 2.07 min
IALSRecommender: Epoch 12 of 30. Elapsed time 2.25 min
IALSRecommender: Epoch 13 of 30. Elapsed time 2.42 min
IALSRecommender: Epoch 14 of 30. Elapsed time 2.62 min
IALSRecommender: Epoch 15 of 30. Elapsed time 2.80 min
IALSRecommender: Epoch 16 of 30. Elapsed time 2.98 min
IALSRecommender: Epoch 17 of 30. Elapsed time 3.18 min
IALSRecommender: Epoch 18 of 30. Elapsed time 3.36 min
IALSRecommende

In [ ]:
#save the baseline MAP in output files
result_df, _ = evaluator_valid.evaluateRecommender(baseline)
with open('baseline.txt', 'w') as f:
    f.write('baseline MAP: {}'.format(result_df.loc[10]["MAP"]))

In [ ]:
from Recommenders.MatrixFactorization.IALSRecommender import IALSRecommender

#try a MF IALS model
recommender_class = IALSRecommender

In [ ]:
import os

output_folder_path = "result_experiments/"

# If directory does not exist, create
if not os.path.exists(output_folder_path):
    os.makedirs(output_folder_path)
    
n_cases = 50
n_random_starts = int(n_cases*0.3)
metric_to_optimize = "MAP"   
cutoff_to_optimize = 10

In [ ]:
from skopt.space import Real, Integer, Categorical
#MF IALS is machine learning-based matrix factorization technique
#the tuning hyperparam are typical of ML models to drive the learning process

hyperparameters_range_dictionary = {
    "num_factors": Integer(10, 50),
    "confidence_scaling": Categorical(["linear", "log"]),
    "alpha": Real(low=1e-2, high=5.0, prior= "log-uniform"),
    "epsilon": Real(low=5e-1, high=5.0, prior= "log-uniform"),
    "reg": Real(low = 1e-5, high = 1e-3, prior = 'log-uniform')
}

In [ ]:
#We also setup the early stopping 
earlystopping_keywargs = {"validation_every_n": 5,
                          "stop_on_validation": True,
                          "evaluator_object": evaluator_valid,
                          "lower_validations_allowed": 5,
                          "validation_metric": metric_to_optimize,
                          }

In [ ]:
from HyperparameterTuning.SearchBayesianSkopt import SearchBayesianSkopt

#create a bayesian optimizer object, we pass the recommender and the evaluator
hyperparameterSearch = SearchBayesianSkopt(recommender_class,
                                         evaluator_validation=evaluator_valid)

In [ ]:
from HyperparameterTuning.SearchAbstractClass import SearchInputRecommenderArgs
  
#provide data needed to create instance of model (one on URM_train, the other on URM_all)
recommender_input_args = SearchInputRecommenderArgs(
    CONSTRUCTOR_POSITIONAL_ARGS = [URM_train],     # For a CBF model simply put [URM_train, ICM_train]
    CONSTRUCTOR_KEYWORD_ARGS = {},
    FIT_POSITIONAL_ARGS = [],
    FIT_KEYWORD_ARGS = earlystopping_keywargs
)

In [ ]:
recommender_input_args_last_test = SearchInputRecommenderArgs(
    CONSTRUCTOR_POSITIONAL_ARGS = [URM_all],
    CONSTRUCTOR_KEYWORD_ARGS = {},
    FIT_POSITIONAL_ARGS = [],
    FIT_KEYWORD_ARGS = earlystopping_keywargs
)

In [ ]:
#import pyximport
#pyximport.install()

In [ ]:
#prepare the environment to run Cython code
#!python run_compile_all_cython.py

In [ ]:
#let's run the bayesian search
hyperparameterSearch.search(recommender_input_args,
                       recommender_input_args_last_test = recommender_input_args_last_test,
                       hyperparameter_search_space = hyperparameters_range_dictionary,
                       n_cases = n_cases,
                       n_random_starts = n_random_starts,
                       save_model = "last",
                       output_folder_path = output_folder_path, # Where to save the results
                       output_file_name_root = recommender_class.RECOMMENDER_NAME, # How to call the files
                       metric_to_optimize = metric_to_optimize,
                       cutoff_to_optimize = cutoff_to_optimize,
                      )

In [ ]:
from Recommenders.DataIO import DataIO

#explore the results of the search
data_loader = DataIO(folder_path = output_folder_path)
search_metadata = data_loader.load_data(recommender_class.RECOMMENDER_NAME + "_metadata.zip")

search_metadata.keys()

In [ ]:
hyperparameters_df = search_metadata["hyperparameters_df"]
hyperparameters_df

In [ ]:
result_on_validation_df = search_metadata["result_on_validation_df"]
result_on_validation_df

In [ ]:
best_hyperparameters = search_metadata["hyperparameters_best"]
best_hyperparameters

This are the best hyperparameters found by the bayesian search, we will use them in our model

In [13]:
#let's fit the model with the hyperparamethers obtained from the previous search and evaluate them on validation set
from Recommenders.MatrixFactorization.IALSRecommender import IALSRecommender

recommender = IALSRecommender(URM_all)
recommender.fit(num_factors = 32, confidence_scaling = "linear", alpha = 0.11916228624446931, epsilon = 0.12, reg = 0.0005078801433259049, epochs = 30.0)
#evaluator_valid.evaluateRecommender(recommender)

IALSRecommender: Epoch 1 of 30.0. Elapsed time 8.32 sec
IALSRecommender: Epoch 2 of 30.0. Elapsed time 16.82 sec
IALSRecommender: Epoch 3 of 30.0. Elapsed time 26.84 sec
IALSRecommender: Epoch 4 of 30.0. Elapsed time 35.40 sec
IALSRecommender: Epoch 5 of 30.0. Elapsed time 43.82 sec
IALSRecommender: Epoch 6 of 30.0. Elapsed time 53.46 sec
IALSRecommender: Epoch 7 of 30.0. Elapsed time 1.03 min
IALSRecommender: Epoch 8 of 30.0. Elapsed time 1.17 min
IALSRecommender: Epoch 9 of 30.0. Elapsed time 1.31 min
IALSRecommender: Epoch 10 of 30.0. Elapsed time 1.47 min
IALSRecommender: Epoch 11 of 30.0. Elapsed time 1.62 min
IALSRecommender: Epoch 12 of 30.0. Elapsed time 1.76 min
IALSRecommender: Epoch 13 of 30.0. Elapsed time 1.90 min
IALSRecommender: Epoch 14 of 30.0. Elapsed time 2.06 min
IALSRecommender: Epoch 15 of 30.0. Elapsed time 2.19 min
IALSRecommender: Epoch 16 of 30.0. Elapsed time 2.33 min
IALSRecommender: Epoch 17 of 30.0. Elapsed time 2.49 min
IALSRecommender: Epoch 18 of 30.0. 

In [ ]:
recommender.save_model(output_folder_path, file_name = recommender.RECOMMENDER_NAME + "_my_own_save.zip" )

# Create final recommendations

In [14]:
test_users = pd.read_csv('../input/recommender-system-2021-challenge-polimi/data_target_users_test.csv')
test_users

,user_id
0,0
1,1
2,2
3,3
4,4
...,...
13645,13645
13646,13646
13647,13647
13648,13648


In [15]:
user_id = test_users['user_id']
recommendations = []
for user in user_id:
    recommendations.append(recommender.recommend(user,cutoff = 10))

In [16]:
for index in range(len(recommendations)):
    recommendations[index]=np.array(recommendations[index])
    
test_users['item_list']= recommendations
test_users['item_list'] = pd.DataFrame([str(line).strip('[').strip(']').replace("'","") for line in test_users['item_list']])
test_users.to_csv('submission.csv', index=False)